In [2]:
%%time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Order 66
%config Completer.use_jedi = False
import warnings
from glob import glob
import sys
warnings.filterwarnings("ignore")
sys.path.append(r"K:\Projects\vscProject\Kaggle\HousePrices")

Wall time: 14 s


In [3]:
%%time

from Script.data import data

X_train, y_train, X_test, y_samp = data()

Tokenization took: 1.00 ms
Type conversion took: 2.00 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 9.99 ms
Type conversion took: 24.99 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 9.99 ms
Type conversion took: 23.99 ms
Parser memory cleanup took: 0.00 ms
Wall time: 5.87 s


In [4]:
print(f"{X_train.shape = }")
print(f"{X_test.shape  = }")

X_train.shape = (1460, 57)
X_test.shape  = (1459, 57)
